In [7]:
import os
import pandas as pd
from PIL import Image
from sklearn.decomposition import PCA
import numpy as np

In [4]:
# Define paths and parameters
data_dir = f"data\extracted_images"  # Set this to your data directory
output_csv = 'output.csv'
image_size = (45, 45)
n_components = 10  # Set to the number of PCA components you want

<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
C:\Users\anhna\AppData\Local\Temp\ipykernel_19124\1237000731.py:2: SyntaxWarning: invalid escape sequence '\e'
  data_dir = f"data\extracted_images"  # Set this to your data directory


In [10]:
# Initialize PCA and DataFrame
pca = PCA(n_components=n_components)
images = []
metadata = []

In [11]:
# Process each folder (label)
for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    if not os.path.isdir(label_path):
        continue
    
    # Process each image in the folder
    for filename in os.listdir(label_path):
        file_path = os.path.join(label_path, filename)
        
        # Open and resize the image, then flatten it
        with Image.open(file_path) as img:
            img = img.resize(image_size).convert('L')  # Resize and convert to grayscale
            img_array = np.array(img).flatten()
            images.append(img_array)  # Collect flattened images for PCA
            
            # Collect metadata for each image
            file_size_kb = os.path.getsize(file_path) / 1024  # File size in KB
            metadata.append([label, filename, file_size_kb])

# Convert images to numpy array
images_array = np.array(images)

# Apply PCA to all images
pca = PCA(n_components=n_components)
pca_features = pca.fit_transform(images_array)

# Combine metadata and PCA features into a single DataFrame
columns = ['Label', 'File Name', 'File Size (KB)'] + [f'PC{i+1}' for i in range(n_components)]
data = [meta + list(features) for meta, features in zip(metadata, pca_features)]
df = pd.DataFrame(data, columns=columns)

# Export to CSV
df.to_csv(output_csv, index=False)
print(f"CSV file saved to {output_csv}")

CSV file saved to output.csv


In [ ]:
# Create DataFrame with column names
columns = ['Label', 'File Name', 'File Size (KB)'] + [f'PC{i+1}' for i in range(n_components)]
df = pd.DataFrame(data, columns=columns)

In [ ]:
# Export to CSV
df.to_csv(output_csv, index=False)
print(f"CSV file saved to {output_csv}")


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score

# Load data

df = pd.read_csv('output.csv')
# Separate features (PC1, PC2, ...) and true labels
X = df.filter(regex='^PC').values  # Selecting all PCA components
true_labels = df['Label'].astype('category').cat.codes  # Encode labels as integers

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize a dictionary to store the evaluation scores
evaluation_scores = {}

# KMeans Clustering
kmeans = KMeans(n_clusters=len(df['Label'].unique()), random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)
evaluation_scores['KMeans'] = {
    'Silhouette Score': silhouette_score(X_scaled, kmeans_labels),
    'ARI': adjusted_rand_score(true_labels, kmeans_labels),
    'NMI': normalized_mutual_info_score(true_labels, kmeans_labels)
}

# Agglomerative Clustering
agglo = AgglomerativeClustering(n_clusters=len(df['Label'].unique()))
agglo_labels = agglo.fit_predict(X_scaled)
evaluation_scores['Agglomerative Clustering'] = {
    'Silhouette Score': silhouette_score(X_scaled, agglo_labels),
    'ARI': adjusted_rand_score(true_labels, agglo_labels),
    'NMI': normalized_mutual_info_score(true_labels, agglo_labels)
}

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)
if len(set(dbscan_labels)) > 1:  # Ensure there is more than one cluster
    evaluation_scores['DBSCAN'] = {
        'Silhouette Score': silhouette_score(X_scaled, dbscan_labels),
        'ARI': adjusted_rand_score(true_labels, dbscan_labels),
        'NMI': normalized_mutual_info_score(true_labels, dbscan_labels)
    }

# Gaussian Mixture Model (GMM)
gmm = GaussianMixture(n_components=len(df['Label'].unique()), random_state=42)
gmm_labels = gmm.fit_predict(X_scaled)
evaluation_scores['GMM'] = {
    'Silhouette Score': silhouette_score(X_scaled, gmm_labels),
    'ARI': adjusted_rand_score(true_labels, gmm_labels),
    'NMI': normalized_mutual_info_score(true_labels, gmm_labels)
}

# Display evaluation scores
for model_name, scores in evaluation_scores.items():
    print(f"{model_name} Evaluation Scores:")
    for metric, score in scores.items():
        print(f"  {metric}: {score:.3f}")
    print("\n")

# Identify the best model based on NMI or ARI (choose based on your preference)
best_model = max(evaluation_scores, key=lambda x: evaluation_scores[x]['NMI'])
print(f"The best model based on NMI is: {best_model}")


MemoryError: Unable to allocate 181. GiB for an array with shape (24265494253,) and data type float64